In [9]:
import pickle
import numpy as np
import pandas as pd
from pathlib import Path

from rica.domain.vibration import vibration
from rica.domain.configure import configure
from rica.use_cases.filter import create_filter_bank, filter_signal
from rica.use_cases.power import (compute_min_max_power, compute_power,
                                  create_power_object)
from rica.domain.density import ExtendedKMeans

In [2]:
def load_model(model_location_name):
    with open(model_location_name, "rb") as fid:
        model = pickle.load(fid)
    if not isinstance(model, ExtendedKMeans):
        print(
            f"There is no extendedkmeans model model in '{model_location_name}' file"
        )
    return model

def save_model(model,location,name):
    to_save=Path(location)/name
    with open(to_save, "wb") as f:
        pickle.dump(model, f)

In [3]:
model=load_model("/home/shankar/GRA/R@/rica/projects/lathrop/day_1_62.5/predict_all_database/oct_14_interpolate_model/model_prev.pkl")
centers=model.cluster_centers_
centers

array([[-1.84851224e-01, -4.89798018e-01, -5.77061766e-01,
        -3.78731923e-01, -1.60061256e-01, -1.64548204e-01,
        -3.78576624e-01],
       [ 7.01526317e-02, -2.50379309e-01, -2.89555622e-01,
        -2.65137608e-01, -6.78580174e-02, -6.67375570e-02,
        -2.58617463e-01],
       [ 2.06610045e-01, -1.25462015e-01, -2.69474845e-01,
        -2.22634328e-01, -6.00976737e-02,  2.32422934e-02,
        -1.96976341e-01],
       [ 2.65164248e-01, -2.04109085e-02, -1.93725641e-04,
        -1.65241652e-01,  4.70025977e-02,  4.21833688e-02,
        -1.41519607e-01],
       [ 3.31508332e-01,  1.10039048e+00,  1.17065688e+00,
         1.49220542e+00,  1.75656032e-01,  1.41728516e-01,
         1.34719890e+00]])

In [4]:
def interpolate(a,b,n):
    each_diff=(b-a)/(n+1)
    betn_points=[type(a)(a+k*each_diff) for k in range(1,n+1)]
    betn_points.insert(0,a)
    betn_points+=[b]
    return betn_points

def transpose_nd_list(l):
    return list(map(list, zip(*l)))

def get_interpolated_centers(centers):
    length=len(centers)
    min_c,max_c=centers[0],centers[-1]
    centers_n=[]
    for a,b in zip(min_c,max_c):
        centers_n+=[interpolate(a,b,length-2)]
    # print(list(np.array(centers_n).T))
    ts_c=transpose_nd_list(centers_n)
    for i in range(len(ts_c)):
        for j in range(len(ts_c[0])):
            centers[i][j]=ts_c[i][j]
    return centers
new_interpolated_centers=get_interpolated_centers(centers)
new_interpolated_centers

array([[-0.18485122, -0.48979802, -0.57706177, -0.37873192, -0.16006126,
        -0.1645482 , -0.37857662],
       [-0.05576133, -0.09225089, -0.1401321 ,  0.08900241, -0.07613193,
        -0.08797902,  0.05286726],
       [ 0.07332855,  0.30529623,  0.29679756,  0.55673675,  0.00779739,
        -0.01140984,  0.48431114],
       [ 0.20241844,  0.70284336,  0.73372722,  1.02447108,  0.09172671,
         0.06515934,  0.91575502],
       [ 0.33150833,  1.10039048,  1.17065688,  1.49220542,  0.17565603,
         0.14172852,  1.3471989 ]])

In [5]:
model.cluster_centers_=new_interpolated_centers

In [6]:
model.cluster_centers_

array([[-0.18485122, -0.48979802, -0.57706177, -0.37873192, -0.16006126,
        -0.1645482 , -0.37857662],
       [-0.05576133, -0.09225089, -0.1401321 ,  0.08900241, -0.07613193,
        -0.08797902,  0.05286726],
       [ 0.07332855,  0.30529623,  0.29679756,  0.55673675,  0.00779739,
        -0.01140984,  0.48431114],
       [ 0.20241844,  0.70284336,  0.73372722,  1.02447108,  0.09172671,
         0.06515934,  0.91575502],
       [ 0.33150833,  1.10039048,  1.17065688,  1.49220542,  0.17565603,
         0.14172852,  1.3471989 ]])

In [7]:
save_model(model,"/home/shankar/GRA/R@/rica/projects/lathrop/day_1_62.5/predict_all_database/oct_14_interpolate_model","model.pkl")

In [8]:
model_=load_model("model.pkl")
centers_=model_.cluster_centers_
centers_

array([[-0.18485122, -0.48979802, -0.57706177, -0.37873192, -0.16006126,
        -0.1645482 , -0.37857662],
       [-0.05576133, -0.09225089, -0.1401321 ,  0.08900241, -0.07613193,
        -0.08797902,  0.05286726],
       [ 0.07332855,  0.30529623,  0.29679756,  0.55673675,  0.00779739,
        -0.01140984,  0.48431114],
       [ 0.20241844,  0.70284336,  0.73372722,  1.02447108,  0.09172671,
         0.06515934,  0.91575502],
       [ 0.33150833,  1.10039048,  1.17065688,  1.49220542,  0.17565603,
         0.14172852,  1.3471989 ]])

In [11]:
pred_ex_k=pd.read_csv("/home/shankar/GRA/R@/rica/projects/lathrop/day_1_62.5/predict_all_database/predict_oct_13_b/processed_data.csv")

In [12]:
pred_inter=pd.read_csv("/home/shankar/GRA/R@/rica/projects/lathrop/day_1_62.5/predict_all_database/oct_14_interpolate_model/processed_data.csv")